In [6]:
import numpy as np
import joblib
from keras.models import load_model

# Load trained model and scaler
model = load_model("model.h5")
scaler = joblib.load("scaler.pkl")

# Manual encodings used during training
gender_map = {"Female": 0, "Male": 1}
geo_map = {"France": 0, "Germany": 1, "Spain": 2}

In [9]:
def predict(features_dict):
  

    # Encode categorical features
    geo_encoded = geo_map[features_dict["Geography"]]
    gender_encoded = gender_map[features_dict["Gender"]]

    # Extract and scale numeric features (order must match training)
    numeric_values = np.array([[
        features_dict["CreditScore"],
        features_dict["Age"],
        features_dict["Tenure"],
        features_dict["Balance"],
        features_dict["NumOfProducts"],
        features_dict["EstimatedSalary"]
    ]])

    numeric_scaled = scaler.transform(numeric_values)[0]

    # Combine all features in correct order
    input_vector = np.array([
        *numeric_scaled,               # 6 scaled numeric features
        geo_encoded,                   # 7: Geography (encoded)
        gender_encoded,                # 8: Gender (encoded)
        features_dict["HasCrCard"],    # 9
        features_dict["IsActiveMember"]# 10
    ])

    # Predict
    probability = model.predict(np.expand_dims(input_vector, axis=0))[0][0]

    label = "Positive" if probability > 0.5 else "Negative"

    return label, float(probability)

In [27]:
if __name__ == "__main__":
    # Example input
    features = {
        "CreditScore": 1000,
        "Age": 40,
        "Tenure": 3,
        "Balance": 60000.0,
        "NumOfProducts": 4,
        "EstimatedSalary": 50000.0,
        "Geography": "France",
        "Gender": "Male",
        "HasCrCard": 1,
        "IsActiveMember": 1
    }

    result, prob = predict(features)
    print(f"Prediction: {result}, Probability: {prob:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Prediction: Negative, Probability: 0.4429


c:\Users\m\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
